In [13]:
import os, sys, argparse, paramiko
from getpass import getpass
from os.path import exists

In [14]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password:  ")

# Host information
host = "192.168.6.71"
port = 2222
username = "sarah.fornaldes"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

In [15]:
sftp = ssh.open_sftp()

# Run a command to scan all known directories on Linux that have a *bin* directory
command = '/home/ubuntu/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin'

ssh.exec_command('kraken')
sftp.put('kraken', 'kraken')

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Get results  from stdout
lines =  stdout.readlines()
#print(lines)

# Convert the list to a string
output =  ''.join(lines)
#print(output)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter
#print(cmd_output)

# Save the cmd_output to a file
with open('results.txt', 'a') as f:
    f.write(cmd_output)

#ssh.close()
sftp.close()

In [16]:
sftp = ssh.open_sftp()

# Uploads results.txt file
uploadFiles = ['results.txt']

# Runs the lsof command, lsof lists all open files for a process                
command = 'sudo -S lsof -i -n'

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Input password to run the command as sudo, including a new line character that provides "Enter" as if you were typing it in on a keyboard
stdin.write(thePass + "\n")

# Get results  from stdout
lines =  stdout.readlines()

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof.txt', 'a') as f:
    f.write(cmd_output)

# End SFTP Session
ssh.close()
sftp.close()

look at threat report, use regex for lsof -p xxxx command
https://github.com/Sarah-F/SYS-320-01/blob/main/Week5/Homework/searchLogCsv.py 